In [5]:
import casadi as ca
import numpy as np
import time
import math

v_e = 0.75
v_p = 0.75
N = 20
dt = 1/N

opti = ca.Opti()

# constraint lambda, mu
lam = opti.variable(3, N)  # index = 1,...,T
mu = opti.variable(1, N)
epsilon = opti.variable(1, N)
C = opti.variable(1, 1)
action = opti.variable(1, 1)

# control variable
u = opti.variable(1, N)
d = opti.variable(1, N)

# state
states = opti.variable(3, N)
x1 = states[0, :]
x2 = states[1, :]
x3 = states[2, :]

# parameters
opt_x0 = opti.parameter(3)

# dynamic function
def f(x_, u_, d_): return ca.vertcat(
     -0.75 + 0.75 * ca.cos(x_[2]) + u_ * ca.sin(x_[1]),
      0.75 * ca.sin(x_[2]) - u_ * x_[0],
      d_ - u_)

def l(x_): return ca.sqrt(x_[0]**2 + x_[1]**2 + 1e-9) - 0.25

# gradient l(x_i) w.r.t x_i
def dldx(x_): return ca.vertcat(
      x_[0]/ca.sqrt(x_[0]**2 + x_[1]**2 + 1e-9),
      x_[1]/ca.sqrt(x_[0]**2 + x_[1]**2 + 1e-9),
      0)

# gradient f(x_i) w.r.t x_i, I + df/dx * dt
def dfdx(x_, u_, dt):
    row1 = ca.horzcat(1, -u_*dt, 0)
    row2 = ca.horzcat(u_*dt, 1, 0)
    row3 = ca.horzcat(-0.75*ca.sin(x_[2])*dt, 0.75*ca.cos(x_[2])*dt, 1)
    return ca.vertcat(row1,
                      row2,
                      row3)

def u_star(lam, x_, dt): return 3*ca.sign((lam[0]*x_[1] - lam[1]*x_[0]-lam[2])*dt)


# init_condition
opti.subject_to(states[:, 0] == opt_x0)

# dynamic constraint
for i in range(N - 1):
    u[:, i] = u_star(lam[:, i + 1], states[:, i], dt)
    x_next = states[:, i] + f(states[:, i], u[:, i], d[:, i]) * dt
    opti.subject_to(states[:, i + 1] == x_next)

# costate constraint
for i in range(1, N - 1):
    opti.subject_to(-mu[:, i]*dldx(states[:, i]) + lam[:, i] -
                    ca.mtimes(dfdx(states[:, i], u[:, i], dt), lam[:, i + 1]) == 0)
opti.subject_to(-mu[:, N - 1] * dldx(states[:, N - 1]) + lam[:, N - 1] == 0)

# multiplier constraint
sum_mu = 0
for i in range(N):
    sum_mu = sum_mu + mu[:, i]
    opti.subject_to(mu[:, i] * (C - l(states[:, i])) == 0)
    opti.subject_to(epsilon[:, i] >= 0)
    opti.subject_to(mu[:, i] >= 0)
    opti.subject_to(C <= l(states[:, i]))
    opti.subject_to(C >= l(states[:, i]) - epsilon[:, i])
opti.subject_to(sum_mu - 1 == 0)


### INITIAL GUESS ###
x1_gus = [0.5, 0.438424627447422, 0.369074705799447, 0.29294769599604625, 0.21113852198328104, 0.12482378011150635, 0.03524493776393854, -0.056309590777699224,
          -0.14852303381775453, -0.2400691394971757, -0.329631192516832, -0.4159210431850402, -0.49769762057167954, -0.573784761288222, -0.6430881186362402,
          -0.7046108937059189, -0.7574682498048215, -0.8008999310771989, -0.8342813035506601, -0.8571322634927412]
x2_gus = [-0.5, -0.5686425857384736, -0.6294201771263511, -0.6814586788587873, -0.7240096128136165, -0.7564610360800551, -0.7783461303099054, -0.7893501572182505,
          -0.789314861416943, -0.7782407556233938, -0.7562870806846811, -0.7237696445279084, -0.6811561073844271, -0.629059335515344, -0.5682286382237293,
          -0.49953898740324815, -0.4239782569058928, -0.3426332533336725, -0.25667390796571504, -0.16733656575476247]
x3_gus = [1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863]


opti.set_initial(x1, x1_gus)
opti.set_initial(x2, x2_gus)
opti.set_initial(x3, x3_gus)

# objective function
obj = -C
for i in range(N):
    obj = obj + epsilon[:, i]

opti.minimize(obj)

# boundrary and control conditions
opti.subject_to(opti.bounded(-1.0, x1, 1.0))
opti.subject_to(opti.bounded(-1.0, x2, 1.0))
opti.subject_to(opti.bounded(-math.pi, x3, math.pi))
opti.subject_to(opti.bounded(-3, d, 3))


ini_state = np.array([[0.5], [-0.5], [1.59]])
opti.set_value(opt_x0, ini_state)

opts_setting = {'ipopt.warm_start_init_point': 'yes', 'ipopt.hessian_approximation': "limited-memory",
                'ipopt.max_iter': 20000, 'ipopt.print_level': 3, 'print_time': 0,
                'ipopt.acceptable_tol': 1e-8, 'ipopt.acceptable_obj_change_tol': 1e-6, 
                'ipopt.inf_pr_output':"internal"}


opti.solver('ipopt', opts_setting)
sol = opti.solve()

res_x = sol.value(states)
res_u = sol.value(u)
res_d = sol.value(d)
res_lambda = sol.value(lam)
res_mu = sol.value(mu)
res_epsilon = sol.value(epsilon)

print(res_x)
print('action u', res_u)
print('action d', res_d)

print()

Total number of variables............................:      201
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      138
Total number of inequality constraints...............:      160
        inequality constraints with only lower bounds:       40
   inequality constraints with lower and upper bounds:       80
        inequality constraints with only upper bounds:       40


Number of Iterations....: 11634

                                   (scaled)                 (unscaled)
Objective...............:   1.6700257144406194e-01    1.6700257144406194e-01
Dual infeasibility......:   2.9770983442367616e-07    2.9770983442367616e-07
Constraint violation....:   7.3428543245768196e-09    7.3428543245768196e-09
Complementarity.........:   5.2053148854840409e-10    5.2053148854840409e-10
Overall NL

In [46]:
opti.debug.value(d)

array([ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3., -3., -3., -3., -3.,
       -3., -3., -3., -3., -3.,  3.,  0.])